### Shared Word

In [35]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [36]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [37]:
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language
word_start = 0  # 0
word_end = 28  # 28 

In [38]:
Path(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

In [39]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [40]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
1005970,karnaya,5
1005971,dörtlümüzün,5
1005972,karnavalınız,5
1005973,hurmanın,5


In [41]:
df_word = df_word_all.iloc[word_start:word_end,]
df_word

,word,frequency


In [42]:
#lang_pair_list = glob.glob(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
#lang_pair_list

In [43]:
#df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,english_word
0,abaküs,abacus
1,abandone,abandon
2,abanoz,ebony
3,abdest,ablution
4,abdesthane,ablution room
...,...,...
2132,zodyak,zodiac
2133,zombi,zombie
2134,zooloji,zoology
2135,zoolojik,zoological


In [44]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [45]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [48]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

In [49]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three", "Four", "Five", "Sentence"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}.xlsx", index=False)    

#### Concat Result

In [50]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,demokrasi
1,tandans
2,fatal
3,komponent
4,doz
...,...
2124,sofistik
2125,teknik
2126,fiber
2127,kompleks


In [51]:
# Option For Frequency
df_word_all = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
1005970,karnaya,5
1005971,dörtlümüzün,5
1005972,karnavalınız,5
1005973,hurmanın,5


In [52]:
# Option
df_word = pd.merge(df_word,df_word_all, how="inner", on="word")
df_word.drop_duplicates(inplace=True)
df_word.reset_index(drop=True, inplace=True)
df_word

,word,frequency
0,demokrasi,4196
1,demokrasi,13
2,fatal,69
3,doz,7168
4,realizm,19
...,...,...
2095,sofistik,41
2096,teknik,32738
2097,fiber,1156
2098,kompleks,873


In [53]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,milyon dolar,6377
1,nükleer bomba,460
2,panik atak,451
3,otomatik pilot,389
4,milyonlarca dolar,317
...,...,...
1926,takım antrenman,6
1927,botanik farmakoloji,6
1928,bebek normal,6
1929,gaz takım,6


In [54]:
# Option For Twogram Sentence
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_twogram_sent = df_twogram_sent.loc[:,["twogram"]]
df_twogram_sent

In [55]:
# Option
df_shared_twogram = pd.merge(df_shared_twogram, df_twogram_sent, how="inner", on="twogram")
df_shared_twogram.drop_duplicates(inplace=True)
df_shared_twogram.reset_index(drop=True, inplace=True)
df_shared_twogram

In [56]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,bravo tango delta,33
1,grup zebra transit,26
2,bebek süper star,24
3,çinli japon amerikalı,23
4,safari motor motel,22
5,alfa tango alfa,22
6,kaliteli normal banka,20
7,kronik psikolojik stres,18
8,fiber optik kablo,18
9,polis robot polis,17


In [57]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram
0,alfa alfa bravo eko,13
1,süpersonik jet turbo sistemli,12
2,radyo televizyon telefon bar,10
3,cd radyo televizyon telefon,10
4,televizyon telefon bar video,10


In [58]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram
0,cd radyo televizyon telefon bar,10
1,radyo televizyon telefon bar video,10


In [59]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,standart prosedür,192
1,alo santral,149
2,fotoğraf çek,145
3,kaptan amerika,138
4,lord kumandan,134
...,...,...
294,manyak polis,6
295,manyak parti,6
296,vampir leydi,6
297,nitrojen oksijen,6


In [60]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,demokrasi,4196,milyon dolar,6377.0,bravo tango delta,33.0,alfa alfa bravo eko,13.0,cd radyo televizyon telefon bar,10.0,standart prosedür,192.0
1,demokrasi,13,nükleer bomba,460.0,grup zebra transit,26.0,süpersonik jet turbo sistemli,12.0,radyo televizyon telefon bar video,10.0,alo santral,149.0
2,fatal,69,panik atak,451.0,bebek süper star,24.0,radyo televizyon telefon bar,10.0,NaN,NaN,fotoğraf çek,145.0
3,doz,7168,otomatik pilot,389.0,çinli japon amerikalı,23.0,cd radyo televizyon telefon,10.0,NaN,NaN,kaptan amerika,138.0
4,realizm,19,milyonlarca dolar,317.0,safari motor motel,22.0,televizyon telefon bar video,10.0,NaN,NaN,lord kumandan,134.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2095,sofistik,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2096,teknik,32738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2097,fiber,1156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2098,kompleks,873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx", index=False)

##### Concat Result With Comma

In [62]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            #word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(10)  # Option
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [63]:
df_word_order_twogram = word_in_twogram(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_twogram(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_twogram(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_twogram(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_twogram(df_shared_all, "word", "sentence") 

In [64]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()

In [65]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [66]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence
0,abanoz,stres abanoz,NaN,NaN,NaN,NaN
1,adrenalin,"doz adrenalin, adrenalin fiziksel, adrenalin s...",NaN,NaN,NaN,NaN
2,adres,"adres telefon, form adres",NaN,NaN,NaN,NaN
3,aerobik,bale aerobik,NaN,NaN,NaN,NaN
4,aerodinamik,aerodinamik iskelet,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
947,sedan,NaN,NaN,NaN,NaN,gri sedan
948,senatör,NaN,NaN,NaN,NaN,telefon senatör
949,tansiyon,NaN,NaN,NaN,NaN,tansiyon normal
950,termometre,NaN,NaN,NaN,NaN,diferansiyel termometre


In [67]:
# Option For Word Frequency
df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
df_word_order_join_all.drop_duplicates(inplace=True)
df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence"]]
df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
df_word_order_join_all.reset_index(drop=True, inplace=True)
df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,kontrol,272328,"trafik kontrol, robot kontrol, rutin kontrol, ...",NaN,NaN,NaN,"rutin kontrol, mikrofon kontrol, kalite kontro..."
1,doktor,266714,"doktor avukat, doktor dans, şans doktor, dokto...",ambulans avukat doktor,NaN,NaN,"pardon doktor, doktor park, bravo doktor, alo ..."
2,polis,247969,"sivil polis, alo polis, profesör polis, metro ...",polis robot polis,NaN,NaN,"alo polis, sivil polis, süper polis, polis mad..."
3,dolar,199343,"milyon dolar, milyonlarca dolar, dolar avans, ...","amerika milyonlarca dolar, milyon dolar transf...",NaN,NaN,"milyonlarca dolar, milyon dolar, milyonlarca d..."
4,vay,162887,"vay rekor, vay termostat, vay süper, vay polis...",NaN,NaN,NaN,"vay süper, vay çakal, vay doktor, vay süper, v..."
...,...,...,...,...,...,...,...
1008,bas,5,"bas gitar, bas çek, bas doktor, bas kovboy, ba...",NaN,NaN,NaN,bas gitar
1009,mart,5,"mart mayıs, mart mayıs",NaN,NaN,NaN,"alo mart, alo mart"
1010,masaj,5,"masaj sauna, i̇spanyol masaj, masaj sauna, i̇s...",NaN,NaN,NaN,NaN
1011,star,5,"süper star, pop star, jet star, star profesyon...","bebek süper star, bebek süper star",NaN,NaN,"süper star, star lord, süper star, star lord"


In [68]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency.xlsx", index=False)

#### Copy Move And Delete

In [52]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*.xlsx")
output_file

['Turkish_English_Shared_Twogram.xlsx',
 'Turkish_English_Shared_Threegram.xlsx',
 'Turkish_English_Shared_Fourgram.xlsx',
 'Turkish_English_Shared_Fivegram.xlsx',
 'Turkish_English_Shared_Sentence.xlsx',
 'Turkish_English_Shared_Result_With_Frequency.xlsx',
 'Turkish_English_Shared_Join_Result_Without_Frequency.xlsx']

In [53]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [54]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass